# Models ensemble by averaging

In general we saw that XGB overall is better (when sampling randomly train and test or in January and February tests), however, Random Forest error does not have a high variance and is better when predicting on March and April, thus we can do an average of the predicted number of sales of every Store in every day, leading to an overall result which is slightly better than the average of the 2 errors.

The other possiblity that we will evaluate is to do an average of the two predictions at a higher level, when we have already grouped by StoreID and Month, this could be even better if one of the model for instance tends to overestimate the predicted value and the over one underestimates it.

**NOTEBOOK GOAL**: Ensemble of models created by averaging the preditions

Averaged predictions:

- **1_RFR** - Notebook 5.3 Random forrest
- **2_XGB** - Notebook 6.4 XGBoost

In [1]:
# reoder variables to work on train or test dataset
predict_for_submission = True
#predict_for_submission = False

In [2]:
from import_man import *
import collections

from BIP import get_BIP_error, apply_BIP_submission_format

### Load predicted tests

**NOTE** If you cannot load the followig datasets, please go to the corresponding notebook and run it to generate the related dataset file. 

In [3]:
dfs_dict = collections.OrderedDict()
# the following dataset will be evaluated

if predict_for_submission:
    dfs_dict['RFR'] = pd.read_csv('./dataset/submission_53_RFR_on_prep.csv')
    dfs_dict['XGB'] = pd.read_csv('./dataset/submission_64_XGBoost_final.csv')
else:
    months='m12'
    dfs_dict['RFR'] = pd.read_csv('./dataset/test_' + months + '_53_RFR_on_prep.csv')
    dfs_dict['XGB'] = pd.read_csv('./dataset/test_' + months + '_64_Model_XGBoost_final.csv')

In [4]:
list(dfs_dict.values())[0].head().T

,0,1,2,3,4
StoreID,1000,1000,1000,1000,1000
Date,01/03/2018,02/03/2018,03/03/2018,05/03/2018,06/03/2018
IsHoliday,0,0,0,0,0
HasPromotions,0,0,0,1,1
NearestCompetitor,326,326,326,326,326
Region,7,7,7,7,7
Region_AreaKM2,9643,9643,9643,9643,9643
Region_GDP,17130,17130,17130,17130,17130
Region_PopulationK,2770,2770,2770,2770,2770
CloudCover,6,7,2,4,4


In [5]:
# let's apply the apply_BIP_submission_format to all the dataframes
if predict_for_submission:
    for mdl_lbl, df in dfs_dict.items():
        dfs_dict[mdl_lbl] = apply_BIP_submission_format(df,real_submit=True)
else:
    for mdl_lbl, df in dfs_dict.items():
        dfs_dict[mdl_lbl] = apply_BIP_submission_format(df)

We decided to work with the average of the already summed up value because it reduces more the error

In [6]:
# make a copy of the first dataframe in order to use it as data structure
df_ens = list(dfs_dict.values())[0].copy()
df_ens['NumberOfSales'] = 0

In [7]:
stores_to_spot = [1000, 1245, 1300, 1301] 

# Let's spot some random rows in order to prove that the average works correctly
if predict_for_submission:
    for mdl_lbl, df in dfs_dict.items():
        print('................................ ' + mdl_lbl + '................................')
        print(df.loc[df.StoreID.isin(stores_to_spot)][['StoreID', 'NumberOfSales']].head(40))
else:
    for mdl_lbl, df in dfs_dict.items():
        print('................................ ' + mdl_lbl + '................................')
        print(df.loc[df.StoreID.isin(stores_to_spot)][['StoreID', 'Target', 'NumberOfSales']].head(40))

................................ RFR................................
     StoreID  NumberOfSales
0       1000         204626
1       1000         187505
490     1245         154612
491     1245         143209
600     1300         129118
601     1300         113058
602     1301         143762
603     1301         124598
................................ XGB................................
     StoreID  NumberOfSales
0       1000         180432
1       1000         170315
490     1245         140838
491     1245         134201
600     1300         111343
601     1300         104699
602     1301         128675
603     1301         121613


In [8]:
if predict_for_submission:
    print(df_ens[['StoreID', 'NumberOfSales']].head())
else:
    print(df_ens[['StoreID', 'Target', 'NumberOfSales']].head())

   StoreID  NumberOfSales
0     1000              0
1     1000              0
2     1001              0
3     1001              0
4     1002              0


In [9]:
# sum up all
for mdl_lbl, df in dfs_dict.items():
    df_ens['NumberOfSales'] += df['NumberOfSales']

# divide by their number
df_ens['NumberOfSales'] /= len(dfs_dict)

if predict_for_submission:
    print(df_ens.loc[df_ens.StoreID.isin(stores_to_spot)][['StoreID', 'NumberOfSales']].head(40))
else:
    print(df_ens.loc[df_ens.StoreID.isin(stores_to_spot)][['StoreID', 'Target', 'NumberOfSales']].head(40))    

     StoreID  NumberOfSales
0       1000       192529.0
1       1000       178910.0
490     1245       147725.0
491     1245       138705.0
600     1300       120230.5
601     1300       108878.5
602     1301       136218.5
603     1301       123105.5


## Write to file

In [10]:
if predict_for_submission:
    df_ens.to_csv('./dataset/submission_82_Ensemble_average.csv', index=False)
else:
    df_ens.to_csv('./dataset/test_' + months + '_82_Ensemble_average.csv', index=False)